## 03-pyspark-window-functions.py

In [0]:
# 03-pyspark-window-functions.py 
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('PySparkExamples').getOrCreate()

simpleData = [("Jaya", "Sales", 3000), ("Mithun", "Sales", 4600),
              ("Rohit", "Sales", 4100), ("Maya", "Finance", 3000),
              ("Jaya", "Sales", 3000), ("Satish", "Finance", 3300),
              ("Joy", "Finance", 3900), ("Jitendra", "Marketing", 3000),
              ("Kumar", "Marketing", 2000), ("Sunu", "Sales", 4100)]
 
columns= ["employee_name", "department", "salary"]

df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|Jaya         |Sales     |3000  |
|Mithun       |Sales     |4600  |
|Rohit        |Sales     |4100  |
|Maya         |Finance   |3000  |
|Jaya         |Sales     |3000  |
|Satish       |Finance   |3300  |
|Joy          |Finance   |3900  |
|Jitendra     |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Sunu         |Sales     |4100  |
+-------------+----------+------+



In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec = Window.partitionBy("department").orderBy("salary")

In [0]:
# ROW_NUMBER() function is a SQL ranking function that assigns a sequential rank number to each new record in a partition. When the SQL Server ROW NUMBER function detects two identical values in the same partition, it assigns different rank numbers to both. The rank number will be determined by the sequence in which they are displayed.

df.withColumn("row_number", row_number().over(windowSpec)) \
    .show(truncate = False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|Maya         |Finance   |3000  |1         |
|Satish       |Finance   |3300  |2         |
|Joy          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jitendra     |Marketing |3000  |2         |
|Jaya         |Sales     |3000  |1         |
|Jaya         |Sales     |3000  |2         |
|Rohit        |Sales     |4100  |3         |
|Sunu         |Sales     |4100  |4         |
|Mithun       |Sales     |4600  |5         |
+-------------+----------+------+----------+



In [0]:
# RANK() function is a window function could be used in SQL Server to calculate a rank for each row within a partition of a result set. The same rank is assigned to the rows in a partition which have the same values. The rank of the first row is 1.
# The ranks may not be consecutive in the RANK() function as it adds the number of repeated rows to the repeated rank to calculate the rank of the next row.

from pyspark.sql.functions import rank
df.withColumn("rank", rank().over(windowSpec)) \
    .show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|         Maya|   Finance|  3000|   1|
|       Satish|   Finance|  3300|   2|
|          Joy|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|     Jitendra| Marketing|  3000|   2|
|         Jaya|     Sales|  3000|   1|
|         Jaya|     Sales|  3000|   1|
|        Rohit|     Sales|  4100|   3|
|         Sunu|     Sales|  4100|   3|
|       Mithun|     Sales|  4600|   5|
+-------------+----------+------+----+



In [0]:
# DENSE_RANK() function assigns a rank to each row within each group of rows with the same name value, based on the following rules:
# 1. Rows with the highest score values are assigned the rank of 1.
# 2. Rows with the next highest score values are assigned the rank of 2, and so on.
# 3. Rows with the same score value are assigned the same rank.

from pyspark.sql.functions import dense_rank
df.withColumn("dense_rank", dense_rank().over(windowSpec)) \
    .show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|         Maya|   Finance|  3000|         1|
|       Satish|   Finance|  3300|         2|
|          Joy|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|     Jitendra| Marketing|  3000|         2|
|         Jaya|     Sales|  3000|         1|
|         Jaya|     Sales|  3000|         1|
|        Rohit|     Sales|  4100|         2|
|         Sunu|     Sales|  4100|         2|
|       Mithun|     Sales|  4600|         3|
+-------------+----------+------+----------+



In [0]:
# PERCENT_RANK() is a window function that calculates the percentile ranking of rows in a result set. The PERCENT_RANK() function returns a percentile ranking number which ranges from zero to one. For a specific row, PERCENT_RANK() uses the following formula to calculate the percentile rank:
# (rank - 1) / (total_rows - 1)

from pyspark.sql.functions import percent_rank
df.withColumn("percent_rank", percent_rank().over(windowSpec)) \
    .show()

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|         Maya|   Finance|  3000|         0.0|
|       Satish|   Finance|  3300|         0.5|
|          Joy|   Finance|  3900|         1.0|
|        Kumar| Marketing|  2000|         0.0|
|     Jitendra| Marketing|  3000|         1.0|
|         Jaya|     Sales|  3000|         0.0|
|         Jaya|     Sales|  3000|         0.0|
|        Rohit|     Sales|  4100|         0.5|
|         Sunu|     Sales|  4100|         0.5|
|       Mithun|     Sales|  4600|         1.0|
+-------------+----------+------+------------+



In [0]:
# NTILE() function is a window function that distributes rows of an ordered partition into a pre-defined number of roughly equal groups. It assigns each group a number_expression ranging from 1.

from pyspark.sql.functions import ntile
df.withColumn("ntile", ntile(2).over(windowSpec)) \
    .show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|         Maya|   Finance|  3000|    1|
|       Satish|   Finance|  3300|    1|
|          Joy|   Finance|  3900|    2|
|        Kumar| Marketing|  2000|    1|
|     Jitendra| Marketing|  3000|    2|
|         Jaya|     Sales|  3000|    1|
|         Jaya|     Sales|  3000|    1|
|        Rohit|     Sales|  4100|    1|
|         Sunu|     Sales|  4100|    2|
|       Mithun|     Sales|  4600|    2|
+-------------+----------+------+-----+



In [0]:
# CUME_DIST() is a window function that calculates the cumulative distribution of value within a set of values. The CUME_DIST() function returns a value that represents the number of rows with values less than or equal to (<= )the current row’s value divided by the total number of rows: i.e. N / total_rows
# where N is the number of rows with the value less than or equal to the current row value.
# and total_rows is the number of rows in the partition or result set being evaluated.

from pyspark.sql.functions import cume_dist    
df.withColumn("cume_dist", cume_dist().over(windowSpec)) \
   .show()

+-------------+----------+------+------------------+
|employee_name|department|salary|         cume_dist|
+-------------+----------+------+------------------+
|         Maya|   Finance|  3000|0.3333333333333333|
|       Satish|   Finance|  3300|0.6666666666666666|
|          Joy|   Finance|  3900|               1.0|
|        Kumar| Marketing|  2000|               0.5|
|     Jitendra| Marketing|  3000|               1.0|
|         Jaya|     Sales|  3000|               0.4|
|         Jaya|     Sales|  3000|               0.4|
|        Rohit|     Sales|  4100|               0.8|
|         Sunu|     Sales|  4100|               0.8|
|       Mithun|     Sales|  4600|               1.0|
+-------------+----------+------+------------------+



In [0]:
# LAG() is a window function that provides access to a row at a specified physical offset which comes before the current row.

from pyspark.sql.functions import lag    
df.withColumn("lag", lag("salary", 2).over(windowSpec)) \
      .show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|         Maya|   Finance|  3000|null|
|       Satish|   Finance|  3300|null|
|          Joy|   Finance|  3900|3000|
|        Kumar| Marketing|  2000|null|
|     Jitendra| Marketing|  3000|null|
|         Jaya|     Sales|  3000|null|
|         Jaya|     Sales|  3000|null|
|        Rohit|     Sales|  4100|3000|
|         Sunu|     Sales|  4100|3000|
|       Mithun|     Sales|  4600|4100|
+-------------+----------+------+----+



In [0]:
# LEAD() is an analytic function that lets you query more than one row in a table at a time without having to join the table to itself. It returns values from the next row in the table.

from pyspark.sql.functions import lead    
df.withColumn("lead", lead("salary", 2).over(windowSpec)) \
    .show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|         Maya|   Finance|  3000|3900|
|       Satish|   Finance|  3300|null|
|          Joy|   Finance|  3900|null|
|        Kumar| Marketing|  2000|null|
|     Jitendra| Marketing|  3000|null|
|         Jaya|     Sales|  3000|4100|
|         Jaya|     Sales|  3000|4100|
|        Rohit|     Sales|  4100|4600|
|         Sunu|     Sales|  4100|null|
|       Mithun|     Sales|  4600|null|
+-------------+----------+------+----+



In [0]:
windowSpecAgg = Window.partitionBy("department")
from pyspark.sql.functions import col, avg, sum, min, max, row_number 

df.withColumn("row", row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row") == 1).select("department", "avg", "sum", "min", "max") \
  .show()

+----------+------+-----+----+----+
|department|   avg|  sum| min| max|
+----------+------+-----+----+----+
|   Finance|3400.0|10200|3000|3900|
| Marketing|2500.0| 5000|2000|3000|
|     Sales|3760.0|18800|3000|4600|
+----------+------+-----+----+----+

